In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


### Gathering Data

In [ ]:
#Read data in a dataframe
df1= pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
df1.head(3)

In [ ]:

url=' https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response=requests.get(url)
print(response.status_code)

In [ ]:
# Save HTML to file

with open("image-predictions.tsv", mode='wb') as file:
    file.write(response.content)

In [ ]:
#Read data in a dataframe
df2= pd.read_table('image-predictions.tsv', sep='\t')

In [ ]:
df2.head(3)

In [ ]:
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'A8OdkrUyojEaUMQ4cOWdTsRE8'
consumer_secret = 'MoKnjOFS0Y4qCQOooPWDmBMUI8kzFv3FfrJCXwXc3aPIoYSMxk'
access_token = '1549735475631489026-K4F7cKx2O5VUbdXl00aQ3tRjG6YGhI'
access_secret = 'kj7jOAXyIfIJLlPbPTwXJrchjwkWdDGse1AiWpWkB1ykL'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = df1.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet-json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepyException as e:  
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)




In [ ]:
df_list = []

with open('tweet-json.txt', 'r') as file:
    for line in file:
        tweet = json.loads(line)
        tweet_id = tweet['id']
        retweet_count = tweet['retweet_count']
        fav_count = tweet['favorite_count']
        df_list.append({'tweet_id':tweet_id,
                       'retweet_count': retweet_count,
                       'favorite_count': fav_count})

In [ ]:
df3= pd.DataFrame(df_list)

In [ ]:
df3.head(3)

# Assesing Data

In this section I will asses the data visually and programmaticaly to identify atleast 8 quality issues and 2 tidiness issues.

##### Visual assesment

In [ ]:
#Assesing a sample of 20 random records of the twitter archived data.
df1.sample(20)


In [ ]:
#Assesing a sample of 20 random records of the image predctions data.
df2.sample(20)

In [ ]:
#Assesing a sample of 20 random records of the twitter json text file data.
df2.sample(20)

##### Programmatic assesment

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
df3.info()

### Quality issues

1. Many missing values in columns: in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp and columns that contain data which is not useful in our Analysis such as Retweets.
2. Incorrect datatypes on the ratings columns specifically the rating denominator.
3. Correcting some unusual names of dogs such as None, a, an, the.
4. Consistency in lower case and upper case naming on records in columns p1, p2, and p3.
5. There are incorrect data types, tweet_id column is supposed to be a str, timestamp column is supposed to be datetime datatype.
6. Wrong values on rating_denominator column different of 10.
7. Outliers on reating_numerator which seems incorrect.
8.  Rating denominator and reating numerator can be one columns

### Tidiness issues

1. Correcting the issue with the dog stages and change the four columns which are  doggo, floofer, pupper,puppo into one columns
2. The tweet_id is duplicated in all three tables which means that they can be converted into one table.


# Cleaning Data

In [ ]:
# Making a copy of the dataframes
df1_copy = df1.copy()
df2_copy = df2.copy()
df3_copy = df3.copy()

##### Define

1. Drop rows in df1_copy table that are retweets.
They are the ones with non-null values on retweeted... columns.
2. Drop columns with more than 75% of missing values such as the in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp columns.

##### Code

In [ ]:
# Droping the retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp columns
columns = ['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp','in_reply_to_status_id', 'in_reply_to_user_id']
df1_copy.drop(columns, axis=1, inplace=True)

##### Test

In [ ]:
df1_copy.sample()

##### Define

1. Fix quality issue of values on rating_denominator different than 10.
2. Fix quality issues with values of rating_numerator that looks too high or weird.
3. Combine rating_numerator and rating_denominator in one column names rating
4. Drop rating_numerator and rating_denominator.

##### 1. Code

In [ ]:
#Check the most common denominator
df1_copy.rating_denominator.value_counts()

In [ ]:
#Filter out where the denominator is 10
df1_copy = df1_copy.query("rating_denominator == 10")
len(df1_copy)

##### 1. Test

In [ ]:
df1_copy.rating_denominator.value_counts()

##### 2.  Code

In [ ]:
#Identify the weird values and filter them out
df1_copy.rating_numerator.value_counts()

# We don't have many values over 15, so I will choose values over 15 are outliers for this feature
len(df1_copy.query("rating_numerator >= 15"))

In [ ]:
# Filtering out the value in the rating_numerator over 15
df1_copy = df1_copy.query("rating_numerator < 15")

##### 2. Test

In [ ]:
# Testing if we have 10 values less than 2333
len(df1_copy)

##### 3. Code

In [ ]:
#Dividing the rating numerator and the rating denominator to make the Trating column.
df1_copy['rating'] = df1_copy['rating_numerator'] / df1_copy['rating_denominator']

In [ ]:
#change the rating column into float
df1_copy['rating'] = df1_copy['rating'].astype(float)

##### 3. Test

In [ ]:
df1_copy.head(3)

##### 4. Code

In [ ]:
#Drop the rating numerator and denominator columns
columns = ['rating_numerator', 'rating_denominator']
df1_copy.drop(columns, axis=1, inplace=True)

##### 4. Test

In [ ]:
df1_copy.head(1)

##### Define

Turning doggo, floofer, pupper and puppo columns into a categorical column called dog_stage

In [ ]:
# Replace the 'None' values by empty spaces

df1_copy.doggo.replace('None', '', inplace=True)
df1_copy.floofer.replace('None', '', inplace=True)
df1_copy.pupper.replace('None', '', inplace=True)
df1_copy.puppo.replace('None', '', inplace=True)

In [ ]:
#Merging the values in one column
df1_copy['dog_stage'] = df1_copy.doggo + df1_copy.floofer + df1_copy.pupper + df1_copy.puppo

In [ ]:
df1_copy['dog_stage'].value_counts()

In [ ]:
# handling multiple stages
df1_copy.loc[df1_copy.dog_stage == 'doggopupper', 'dog_stage'] = 'doggo, pupper'
df1_copy.loc[df1_copy.dog_stage == 'doggopuppo', 'dog_stage'] = 'doggo, puppo'
df1_copy.loc[df1_copy.dog_stage == 'doggofloofer', 'dog_stage'] = 'doggo, floofer'

In [ ]:
# Dropping the doggo, floofer, pupper, puppo columns
columns = ['doggo','floofer','pupper','puppo']
df1_copy.drop(columns, axis=1, inplace=True)

In [ ]:
# handle missing values in dog_stage column
df1_copy.loc[df1_copy.dog_stage == '', 'dog_stage'] = np.nan

##### Test

In [ ]:
df1_copy.sample(8)

##### Define

Merge df3_copy and df2_copy dataframes  which are the data from the twitter json txt file and the data of the image predictions  into df1_copy.

##### Code

In [ ]:
# Merging df3_copy with df1_copy. 
df1_copy = pd.merge(df1_copy, df3_copy, on=['tweet_id'])

In [ ]:
# Merging df3_copy with df1_copy
# Use inner join because we want to keep only the tweets with images
df1_copy = pd.merge(df1_copy, df2_copy, on=['tweet_id'])

##### Test

In [ ]:
df1_copy.info()

##### Define

Fix the Datatype of timestamp and the tweet_id

##### Code

In [ ]:
#timestamp
df1_copy['timestamp'] = pd.to_datetime(df1_copy['timestamp'])

#tweet_id
df1_copy['tweet_id'] = df1_copy['tweet_id'].astype(str)


##### Test

In [ ]:
df1_copy.dtypes

##### Define

Turn the data in columns p1, p2, and p3 to lowercase

In [ ]:
df1_copy['p1']=df1_copy['p1'].str.lower()
df1_copy['p2']=df1_copy['p2'].str.lower()
df1_copy['p3']=df1_copy['p3'].str.lower()

##### Test

In [ ]:
df1_copy.sample(10)

##### Define

1. Identify wrong name of dogs such as 'his', 'a', 'very', 'unacceptable', 'such','the', 'an', among others.
2. Replace these values by NaN so they are defined as unknown values.


##### Code

In [ ]:
df1_copy.name.value_counts()

In [ ]:
df1_copy['name'] = df1_copy['name'].replace(['None', 'a', 'an', 'the','by','my',], 'No name')

##### Test

In [ ]:
df1_copy.name.sample(10)

# Storing clean data

In [ ]:
df1_copy.to_csv('twitter_archive_master.csv', index=False)

# Analysis and Visualisation

In this section, analyze and visualize your wrangled data. You must produce at least three (3) insights and one (1) visualization.

In [ ]:
#imports
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
#Reading the dataset
df=pd.read_csv('twitter_archive_master.csv')
df.head()

In [ ]:
#main statistics
df.describe()

### Questions

##### What is the relationship between dog_stage and people's interactions either in Likes and Retweets  ?

In [ ]:
#first we will group the dog_stage data with respect to retweet count and then save it in a new list create a new series 
df_dog_stage = df.groupby('dog_stage')['retweet_count'].count()
df_dog_stage

For this visualization i will be making use of the series just created above to create a bar chart to analyze the rate of dog stages to retweet count.

In [ ]:
plt.figure(figsize=(10, 8))
plt.bar(np.arange(7), df_dog_stage, color = 'Green')
plt.xticks(np.arange(7), ('doggo', 'doggo, floofer ', 'doggo, pupper', 'doggo, puppo', 'floofer', 'pupper','puppo'))
plt.xlabel('Dog Stages')
plt.ylabel('Retweets counts')
plt.title('Dog stages and their Retweets counts')
plt.show()

In [ ]:
dog_stage =['doggo','doggo,floofer','doggo,pupper','doggo,puppo ','floofer','pupper','puppo']
df1_dog_stage= df.groupby('dog_stage')['favorite_count'].count()
df1_dog_stage.plot(kind='bar',title= 'Dog stages and their favorites count',color='Green', figsize=(9,6))
plt.xlabel=('Dog_stage')
plt.ylabel=('Favorite_count')

##### Insights

* The dog stage with the highest retweets and likes is on  the pupper stage.
* doggo,floofer and doggo,puppo stages are the least to retweeted and liked.
* Lastly, the dogs at pupper stages are the most famous since they are the most retweeted and liked.
